<a href="https://colab.research.google.com/github/shubhamt2897/DL_BB/blob/main/DL_Bounding_Box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade torch torchvision
!pip install numpy matplotlib albumentations kornia open3d gdown



In [ ]:
# Step 2: Configure Git
!git config --global user.email "sp.tangadpalliwar@stud.fh-sm.de "  # Replace with your email
!git config --global user.name "shubhamt2897"

In [ ]:
!pip install gdown


In [ ]:
import os

dataset_path = "/content/dl_challenge"
print(os.listdir(dataset_path))  # List available files/folders


In [ ]:

!gdown "https://drive.google.com/uc?id=11s-GLb6LZ0SCAVW6aikqImuuQEEbT_Fb"

Downloading...
From (original): https://drive.google.com/uc?id=11s-GLb6LZ0SCAVW6aikqImuuQEEbT_Fb
From (redirected): https://drive.google.com/uc?id=11s-GLb6LZ0SCAVW6aikqImuuQEEbT_Fb&confirm=t&uuid=3b9443b1-1a4e-4b9b-a444-af619f5ab314
To: /content/dl_challenge.tar.xz
100% 2.44G/2.44G [00:30<00:00, 78.8MB/s]


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import os
import json
from torchvision import transforms

class Custom3DDataset(Dataset):
    def __init__(self, img_dir, bbox_file, transform=None):
        self.img_dir = img_dir
        self.bbox_file = bbox_file
        self.transform = transform
        self.images = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')])
        self.bboxes = self.load_bboxes(bbox_file)

    def load_bboxes(self, bbox_file):
        with open(bbox_file, 'r') as f:
            bbox_data = json.load(f)
        return bbox_data

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)

        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get corresponding bbox
        bbox = self.bboxes[img_name] if img_name in self.bboxes else None

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(bbox, dtype=torch.float32) if bbox is not None else None